In [1]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from starfish.core.structured_llm import StructuredLLM
from starfish.utils.data_factory import data_factory

In [3]:
facts_generator = StructuredLLM(
                                model_name="openai/gpt-4o-mini",
                                prompt="""Generate facts about {{city_name}} on {{topic}}""" , 
                                output_schema=[{'name': 'question', 'type': 'str'}, {'name': 'answer', 'type': 'str'}],
                                model_kwargs={"temperature": 0.7}
                            )
response = await facts_generator.run(city_name="San Francisco", topic="history")
response.data 

[{'question': "What significant event in San Francisco's history took place in 1906?",
  'answer': 'The Great San Francisco Earthquake occurred on April 18, 1906, causing widespread destruction and leading to a massive fire that devastated much of the city.'}]

In [1]:
user_instructions = """generate facts about san francisco"""
num_records = 100
topics = {'history': 10, 'culture': 10, 'food': 10}

data_market.run(name = 'generator_with_topic', 
                   user_instructions = """generate facts about san francisco""",
                   num_records = 100,
                   topics = {'history': 10, 'culture': 10, 'food': 10})

In [3]:

from starfish.core.prebuilt import prepare_topic
topic_list = await prepare_topic(num_records=num_records, user_instructions=user_instructions, records_per_topic=10) #  topics = topics,


In [7]:
len(topic_list)

100

In [6]:

@data_factory(max_concurrency=200)
async def generate_facts(user_instructions: str, topic: str):
    facts_generator = StructuredLLM(
                                model_name="openai/gpt-4o-mini",
                                prompt="""{{user_instructions}} on {{topic}}""" , 
                                output_schema=[{'name': 'question', 'type': 'str'}, {'name': 'answer', 'type': 'str'}],
                                model_kwargs={"temperature": 0.7}
                            )
    response = await facts_generator.run(user_instructions=user_instructions, topic=topic)
    return response.data 

facts_data = generate_facts.run(data = topic_list, user_instructions=user_instructions)

In [10]:

len(facts_data)

100